In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

#Activation Function

In [ ]:
# Acivation Function
t
class ReLU(Layer):    
    def __init__(self, output_dim):
        self.units = output_dim
        self.type = 'ReLU'
 
    def __str__(self):
        return f"{self.type} Layer"       
         
    def forward(self, input_val):
        self._prev_acti = np.maximum(0, input_val)
        return self._prev_acti
     
    def backward(self, dJ):
        return dJ * np.heaviside(self._prev_acti, 0)


class Sigmoid(Layer):
    def __init__(self, output_dim):
        self.units = output_dim
        self.type = 'Sigmoid'
 
    def __str__(self):
        return f"{self.type} Layer"       
         
    def forward(self, input_val):
        self._prev_acti = 1 / (1 + np.exp(-input_val))
        return self._prev_acti
     
    def backward(self, dJ):
        sig = self._prev_acti
        return dJ * sig * (1 - sig)

class Softmax(Layer):
    def __init__(self, output_dim):
        self.units = output_dim
        self.type = 'Softmax'

    def softmax(a):
      C = np.max(a)
      exp_a = np.exp(a - C)
      if a.ndim == 1:
          sum_exp_a = np.sum(exp_a)
          y = exp_a / sum_exp_a
      else:
          sum_exp_a = np.sum(exp_a, 1)
          sum_exp_a = sum_exp_a.reshape(sum_exp_a.shape[0], 1)
          y = exp_a / sum_exp_a
      return y

    def cross_entropy_loss(y, t):
    C = 1e-7
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    batch_size = y.shape[0]
    return -np.sum(t * np.log(y + C)) / batch_size
    
    def forward(self, input_val, t):
        self.t = t
        self._prev_acti = softmax(input_val)
        self.loss = cross_entropy_loss(self.y, self.t)
        return self.loss

    def backward(self):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        return dx

class Linear(Layer):
    def __init__(self, input_dim, output_dim):
        self.weights = np.random.rand(output_dim, input_dim)
        self.biases = np.random.rand(output_dim, 1)
        self.type = 'Linear'
 
    def __str__(self):
        return f"{self.type} Layer"
         
    def forward(self, input_val):
        self._prev_acti = input_val
        return np.matmul(self.weights, input_val) + self.biases
     
    def backward(self, dA):
        dW = np.dot(dA, self._prev_acti.T)
        dB = dA.mean(axis=1, keepdims=True)
         
        delta = np.dot(self.weights.T, dA)
         
        return delta, dW, dB
     
    def optimize(self, dW, dB, rate):
        self.weights = self.weights - rate * dW
        self.biases = self.biases - rate * dB

#Loss Function

In [ ]:
# Loss Function
class MeanSquaredError(Layer):
    def __init__(self, predicted, real):
        self.predicted = predicted
        self.real = real
        self.type = 'Mean Squared Error'
     
    def forward(self):
        return np.power(self.predicted - self.real, 2).mean()
 
    def backward(self):
        return 2 * (self.predicted - self.real).mean()

class BinaryCrossEntropy(Layer):
    def __init__(self, predicted, real):
        self.real = real
        self.predicted = predicted
        self.type = 'Binary Cross-Entropy'
     
    def forward(self):
        n = len(self.real)
        loss = np.nansum(-self.real * np.log(self.predicted) - (1 - self.real) * np.log(1 - self.predicted)) / n
         
        return np.squeeze(loss)
     
    def backward(self):
        n = len(self.real)
        return (-(self.real / self.predicted) + ((1 - self.real) / (1 - self.predicted))) / n



#Multi Layer Perceptron Model (Sigmoid)

In [ ]:
import numpy as np

# Activation Function (Sigmoid)
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def Backpropagation(f, input_data): #numerical_derivative 수치 미분 함수
    delta_x = 1e-4

    ret = np.zeros_like(input_data)
    it = np.nditer(input_data, flags=['multi_index'])

    while not it.finished:
        idx = it.multi_index

        tmp = input_data[idx]
        input_data[idx] = float(tmp) + delta_x
        fx1 = f(input_data)

        input_data[idx] = float(tmp) - delta_x
        fx2 = f(input_data)

        ret[idx] = (fx1 - fx2) / (2 * delta_x)
        input_data[idx] = tmp
        it.iternext()

    return ret

class MLPmodel:
  def __init__(self, model_name, x, t):
    self.name = model_name

    #input data
    self.__x = x
    self.__t = t

    #weight, bias
    i =  [[ 2.63259021,  2.44820463 , 4.63134428 , 1.17225176 , 5.48438174,  7.2549209 ],[ 3.28485616 ,-1.69787055,  5.22847983 , 2.02503399 , 5.40761403  ,6.48606646]]
    self.__W = np.array(i)
    j = [-4.61606264  ,1.46435451, -7.54664825 ,-2.47371219 ,-2.32749526, -3.09309171]
    self.__b = np.array(j)
    k = [[ -6.64307767],[ -4.60866687],[-11.09625077],[ -4.26531046],[  8.01868687],[ 11.17017424]]
    self.__W2 = np.array(k)
    g = [-4.44898273]
    self.__b2 = np.array(g)

    #learning rate, loss function
    self.__learning_rate = 1e-1
    self.loss_func = self.__feed_forward


  def __feed_forward(self):
    delta = 1e-7

    #input -> hidden
    y = np.dot(self.__x, self.__W) + self.__b
    y_hat = sigmoid(y) # Activation Function
    #hidden -> output
    y2 = np.dot(y_hat, self.__W2) + self.__b2
    y2_hat = sigmoid(y2)

    #CrossEntropy로 Error detection
    return -np.sum(self.__t * np.log(y2_hat + delta) + (1 - self.__t) * np.log(1 - y2_hat + delta))

  

  def train(self):
    f = lambda x: self.__feed_forward()

    for step in range(8001):
            self.__W -= self.__learning_rate * Backpropagation(f, self.__W)
            self.__b -= self.__learning_rate * Backpropagation(f, self.__b)
            self.__W2 -= self.__learning_rate * Backpropagation(f, self.__W2)
            self.__b2 -= self.__learning_rate * Backpropagation(f, self.__b2)

  def predict(self, x):
    #input -> hidden
    y = np.dot(x, self.__W) + self.__b
    y_hat = sigmoid(y) # Activation Function
    #hidden -> output
    y2 = np.dot(y_hat, self.__W2) + self.__b2
    y2_hat = sigmoid(y2)

    if y2_hat < 0.5:
            return 0, y2_hat
    return 1, y2_hat

    #if y2_hat <0.5 -> 0출력, 아니면 1출력 -> sigmoid 출력

In [ ]:
#data input
x = np.array([[0, 0], [0, 1], [1, 0], [1, 1]]).reshape([4,2])
y = np.array([0,1,1,0]).reshape([4,1])

#train
MLP = MLPmodel("MLP", x, y)
MLP.train()

#output
print(MLP.predict([0,0]))
print(MLP.predict([1,0]))
print(MLP.predict([0,1]))
print(MLP.predict([1,1]))

#Multi Layer Perceptron (Softmax)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

In [ ]:
def softmax(x):
    c = np.max(x)
    minus = x-c
    exp_b = np.exp(minus)
    sum_exp_b = np.sum(exp_b)
    return exp_b /sum_exp_b


k = [0.3,2,0.3]

print(softmax(k))

In [ ]:
y3_hat = [0.3,0.6,0.4]
k = np.max(y3_hat, axis=0)
i = 0
for i in range(2):
  if y3_hat[i] == k:
    y3_hat[i] = 1
print(y3_hat)


In [ ]:
y3_hat = [0.3,0.6,0.4]

k = np.max(y3_hat, axis=0)
i = 0
for i in range(2):
  if y3_hat[i] == k:
        y3_hat[i] = 1
if(y3_hat[0]==1):
  print(1)
if(y3_hat[1]==1):
  print(2)
if(y3_hat[2]==1):
  print(3)

In [ ]:
import numpy as np
np.set_printoptions(precision=15)
def softmax(x):
    exp_a = np.exp(x)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    return y

y3 = [[67.34759010386061 ,60.49088997410166 ,61.34620329299571]]
y3_hat = softmax(y3)
print(y3_hat)

In [ ]:
import numpy as np

# Activation Function (Sigmoid)
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def softmax(x):
    c = np.max(x)
    minus = x-c
    exp_b = np.exp(minus)
    sum_exp_b = np.sum(exp_b)
    return exp_b /sum_exp_b

def relu(x):
    return(np.maximum(0,x))

def Leaky_ReLU(x):
  return (np.maximum(0.01*x,x))

def Backpropagation(f, input_data): #numerical_derivative 수치 미분 함수
    delta_x = 1e-4

    ret = np.zeros_like(input_data)
    it = np.nditer(input_data, flags=['multi_index'])

    while not it.finished:
        idx = it.multi_index

        tmp = input_data[idx]
        input_data[idx] = float(tmp) + delta_x
        fx1 = f(input_data)

        input_data[idx] = float(tmp) - delta_x
        fx2 = f(input_data)

        ret[idx] = (fx1 - fx2) / (2 * delta_x)
        input_data[idx] = tmp
        it.iternext()

    return ret

class MLPmodel:
  
  def __init__(self, model_name, x_data, t):
    self.name = model_name

    #input data
    self.__x = x_data
    self.__t = t

    #weight, bias
    i =  [[ 2.63259021,  2.44820463 , 4.63134428 , 1.17225176 , 5.48438174,  7.2549209 ],[ 3.28485616 ,-1.69787055,  5.22847983 , 2.02503399 , 5.40761403  ,6.48606646]]
    self.__W = np.random.rand(4,12)
    j = [-4.61606264  ,1.46435451, -7.54664825 ,-2.47371219 ,-2.32749526, -3.09309171]
    self.__b = np.random.rand(1,12)
    k = [[ -6.64307767],[ -4.60866687],[-11.09625077],[ -4.26531046],[  8.01868687],[ 11.17017424]]
    self.__W2 = np.random.rand(12,12)
    g = [-4.44898273]
    self.__b2 = np.random.rand(1,12)
    self.__W3 = np.random.rand(12,3)
    self.__b3 = np.random.rand(1,3)

    #learning rate, loss function
    self.__learning_rate = 0.01
    self.loss_func = self.__feed_forward

  
  def __feed_forward(self):
    delta = 1e-7

    #input -> hidden
    y = np.dot(self.__x, self.__W) + self.__b
    y_hat = Leaky_ReLU(y) # Activation Function
    #hidden -> hidden2
    y2 = np.dot(y_hat, self.__W2) + self.__b2
    y2_hat = Leaky_ReLU(y2)
    #hidden2 -> output
    y3 = np.dot(y2_hat, self.__W3) + self.__b3
    y3_hat = softmax(y3)

    import tensorflow as tf
    from tensorflow import keras
    cce = tf.keras.losses.CategoricalCrossentropy()
    loss_cce  = cce(self.__t,   y3_hat)
    return loss_cce

  

  def train(self):
    f = lambda x: self.__feed_forward()

    for step in range(8001):
            self.__W -= self.__learning_rate * Backpropagation(f, self.__W)
            self.__b -= self.__learning_rate * Backpropagation(f, self.__b)
            self.__W2 -= self.__learning_rate * Backpropagation(f, self.__W2)
            self.__b2 -= self.__learning_rate * Backpropagation(f, self.__b2)
            self.__W3 -= self.__learning_rate * Backpropagation(f, self.__W3)
            self.__b3 -= self.__learning_rate * Backpropagation(f, self.__b3)

  def predict(self, x_data):
    #input -> hidden
    print(x_data)
    y = np.dot(x_data, self.__W) + self.__b
    print(self.__W)
    y_hat = Leaky_ReLU(y) # Activation Function
    print(y_hat)
    
    #hidden -> hidden2
    y2 = np.dot(y_hat, self.__W2) + self.__b2
    y2_hat = Leaky_ReLU(y2)
    print(y2_hat)
    #hidden2 -> output
    y3 = np.dot(y2_hat, self.__W3) + self.__b3
    y3_hat = softmax(y3)
    print(y3_hat)
    for i in range(0,3):
      if y3_hat[0][i]==max(y3_hat[0]):
        return i+1


    #if y2_hat <0.5 -> 0출력, 아니면 1출력 -> sigmoid 출력

In [ ]:
# output one-hot-encoding
from sklearn import preprocessing
import pandas as pd
data = pd.read_csv('/content/gdrive/MyDrive/kaggle/y.csv')

le = preprocessing.LabelEncoder()

data_2 = data.apply(le.fit_transform)

enc = preprocessing.OneHotEncoder()

enc.fit(data_2)

onehotlabels = enc.transform(data_2).toarray()

#data input
from numpy import genfromtxt
x = genfromtxt('/content/gdrive/MyDrive/kaggle/x.csv', delimiter=',', skip_header = 1)
y = onehotlabels
#train
MLP = MLPmodel("MLP", x, y)
MLP.train()

In [ ]:
# print(MLP.predict([5.1],[3.5],[1.4],[0.2]]))
for i in range(149):
  print(MLP.predict(x[i]))




In [ ]:
from numpy import genfromtxt
data = genfromtxt('/content/gdrive/MyDrive/kaggle/y.csv', delimiter=',', skip_header = 1)
print(onehotlabels)

In [ ]:
from sklearn import preprocessing
import pandas as pd
data = pd.read_csv('/content/gdrive/MyDrive/kaggle/y.csv')

le = preprocessing.LabelEncoder()

data_2 = data.apply(le.fit_transform)

enc = preprocessing.OneHotEncoder()

enc.fit(data_2)

onehotlabels = enc.transform(data_2).toarray()
onehotlabels


#MLP 2

In [ ]:
import numpy as np
class NeuralNetwork(object):
    def __init__(self, layers = [2 , 10, 1], activations=['sigmoid', 'sigmoid']):
        assert(len(layers) == len(activations)+1)
        self.layers = layers
        self.activations = activations
        self.weights = []
        self.biases = []
        for i in range(len(layers)-1):
            self.weights.append(np.random.randn(layers[i+1], layers[i]))
            self.biases.append(np.random.randn(layers[i+1], 1))
    
    def feedforward(self, x):
        # return the feedforward value for x
        a = np.copy(x)
        z_s = []
        a_s = [a]
        for i in range(len(self.weights)):
            activation_function = self.getActivationFunction(self.activations[i])
            z_s.append(self.weights[i].dot(a) + self.biases[i])
            a = activation_function(z_s[-1])
            a_s.append(a)
        return (z_s, a_s)
    def backpropagation(self,y, z_s, a_s):
        dw = []  # dC/dW
        db = []  # dC/dB
        deltas = [None] * len(self.weights)  # delta = dC/dZ  known as error for each layer
        # insert the last layer error
        deltas[-1] = ((y-a_s[-1])*(self.getDerivitiveActivationFunction(self.activations[-1]))(z_s[-1]))
        # Perform BackPropagation
        for i in reversed(range(len(deltas)-1)):
            deltas[i] = self.weights[i+1].T.dot(deltas[i+1])*(self.getDerivitiveActivationFunction(self.activations[i])(z_s[i]))        
        #a= [print(d.shape) for d in deltas]
        batch_size = y.shape[1]
        db = [d.dot(np.ones((batch_size,1)))/float(batch_size) for d in deltas]
        dw = [d.dot(a_s[i].T)/float(batch_size) for i,d in enumerate(deltas)]
        # return the derivitives respect to weight matrix and biases
        return dw, db
    def train(self, x, y, batch_size=10, epochs=100, lr = 0.01):
# update weights and biases based on the output
        for e in range(epochs): 
            print("epoch :",e+1,'\n')
            i=0
            while(i<len(y)):
                x_batch = x[i:i+batch_size]
                y_batch = y[i:i+batch_size]
                i = i+batch_size
                z_s, a_s = self.feedforward(x_batch)
                dw, db = self.backpropagation(y_batch, z_s, a_s)
                self.weights = [w+lr*dweight for w,dweight in  zip(self.weights, dw)]
                self.biases = [w+lr*dbias for w,dbias in  zip(self.biases, db)]
                print("loss = {}".format(np.linalg.norm(a_s[-1]-y_batch) ))
    @staticmethod
    def getActivationFunction(name):
        if(name == 'sigmoid'):
            return lambda x : np.exp(x)/(1+np.exp(x))
        elif(name == 'linear'):
            return lambda x : x
        elif(name == 'relu'):
            def relu(x):
                y = np.copy(x)
                y[y<0] = 0
                return y
            return relu
        else:
            print('Unknown activation function. linear is used')
            return lambda x: x
    
    @staticmethod
    def getDerivitiveActivationFunction(name):
        if(name == 'sigmoid'):
            sig = lambda x : np.exp(x)/(1+np.exp(x))
            return lambda x :sig(x)*(1-sig(x)) 
        elif(name == 'linear'):
            return lambda x: 1
        elif(name == 'relu'):
            def relu_diff(x):
                y = np.copy(x)
                y[y>=0] = 1
                y[y<0] = 0
                return y
            return relu_diff
        else:
            print('Unknown activation function. linear is used')
            return lambda x: 1

In [ ]:
if __name__=='__main__':
    import matplotlib.pyplot as plt
    nn = NeuralNetwork([1, 100, 1],activations=['sigmoid', 'sigmoid'])
    X = 2*np.pi*np.random.rand(3000).reshape(1, -1)
    y = np.sin(X)
    
    nn.train(X, y, epochs=1000, batch_size=64, lr = .1)
    _, a_s = nn.feedforward(X)
    #print(y, X)
    plt.scatter(X.flatten(), y.flatten())
    plt.scatter(X.flatten(), a_s[-1].flatten())
    plt.show()

#CNN model

In [ ]:
import numpy as np

class CNNmodel:

  def __init__(self, x, hidden, y):
        self.__x = x
        self.__hidden = hidden
        self.__y = y

        self.__learning_rate = 1e-3

        self.__W = np.random.rand(x, hidden)
        self.__b = np.random.rand(hidden)

        self.__W2 = np.random.rand(hidden, y)
        self.__b2 = np.random.rand(y)

        self.__data = np.array([])
        self.__one_hot = np.array([])

  # Activation Function (Sigmoid)
  def sigmoid(x):
      return 1 / (1 + np.exp(-x))

  def Backpropagation(f, input_data): #numerical_derivative 수치 미분 함수
    delta_x = 1e-4

    ret = np.zeros_like(input_data)
    it = np.nditer(input_data, flags=['multi_index'])

    while not it.finished:
        idx = it.multi_index

        tmp = input_data[idx]
        input_data[idx] = float(tmp) + delta_x
        fx1 = f(input_data)

        input_data[idx] = float(tmp) - delta_x
        fx2 = f(input_data)

        ret[idx] = (fx1 - fx2) / (2 * delta_x)
        input_data[idx] = tmp
        it.iternext()

    return ret

  # 순전파
  def __feed_forward(self):
        delta = 1e-7

        y = np.dot(self.__x, self.__W) + self.__b
        y_hat = sigmoid(y)

        y2 = np.dot(y_hat, self.__W2) + self.__b2
        y2_hat = sigmoid(y2)

        return -np.sum(self.__one_hot * np.log(y2_hat + delta) + (1 - self.__one_hot) * np.log(1 - y2_hat + delta))

  def loss_val(self):
        delta = 1e-7

        y = np.dot(self.__x, self.__W) + self.__b
        y_hat = sigmoid(y)

        y2 = np.dot(y_hat, self.__W2) + self.__b2
        y2_hat = sigmoid(y2)

        return -np.sum(self.__one_hot * np.log(y2_hat + delta) + (1 - self.__one_hot) * np.log(1 - y2_hat + delta))

  # 예측값
  def predict(self, x):
    #input -> hidden
    y = np.dot(self.__x, self.__W) + self.__b
    y_hat = sigmoid(y) # Activation Function
    #hidden -> output
    y2 = np.dot(y_hat, self.__W2) + self.__b2
    y2_hat = sigmoid(y2)
    return np.argmax(y2_hat)

  # 정확도
  def accuracy(self, test_data):
        count = 0
        for index in range(len(test_data)):
            label = int(test_data[index, 0])
            data = test_data[index, 1:]
            predicted_val = self.predict(data)
            if predicted_val == label:
                count += 1

        print('Accuracy = ', 100 * (count / float(len(test_data))))

  def train(self, train_data):
        label = int(train_data[0])
        self.__data = (train_data[1:] / 255.0 * 0.99) + 0.01
        self.__one_hot = np.zeros(self.__y) + 0.01
        self.__one_hot[label] = 0.99

        f = lambda x: self.__feed_forward()
        self.__W -= self.__learning_rate * Backpropagation(f, self.__W)
        self.__b -= self.__learning_rate * Backpropagation(f, self.__b)
        self.__W2 -= self.__learning_rate * Backpropagation(f, self.__W2)
        self.__b2 -= self.__learning_rate * Backpropagation(f, self.__b2)


In [ ]:
import pandas as pd

train_data = pd.read_csv('/content/gdrive/MyDrive/kaggle/Iris.csv')
test_data = pd.read_csv('/content/gdrive/MyDrive/kaggle/test.txt')

mnist = CNNmodel(784, 50, 10)
for step in range(30001):
    mnist.train(train_data[step])

    if step % 400 == 0:
        print('step = ', step, 'loss_val = ', mnist.loss_val())

mnist.accuracy(test_data)